# Imports

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import math

from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import optuna

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
X_train = np.load('data/x_train.npy')
y_train = np.load('data/y_train.npy')

X_test = np.load('data/x_test.npy')
y_test = np.load('data/y_test.npy')

In [7]:
from sklearn.model_selection import train_test_split

X = np.concatenate([X_train, X_test], axis=0)
y = np.concatenate([y_train, y_test], axis=0)

X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.2, random_state=42, stratify=y_trainval)

# DT Feature Selection

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import f1_score
import time

In [9]:
start_time = time.time()

clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

print(f"Seconds: {time.time() - start_time}")

Seconds: 13.777572631835938


In [10]:
feature_importances = clf.feature_importances_
feature_importances = np.array(feature_importances)

feature_importances

array([6.00004054e-04, 2.32275550e-01, 5.81909506e-02, 3.15756934e-03,
       4.25585385e-03, 1.86017758e-02, 0.00000000e+00, 1.12297756e-02,
       2.06835319e-05, 9.79839366e-03, 1.15726586e-01, 1.34670186e-02,
       0.00000000e+00, 0.00000000e+00, 1.43275550e-02, 2.18915803e-02,
       3.91414866e-04, 4.75027837e-03, 3.65356642e-02, 1.32100357e-02,
       5.00913271e-04, 3.91093578e-03, 5.11511835e-03, 7.93200920e-04,
       2.34232585e-02, 1.96728413e-03, 1.04062365e-02, 2.28049606e-03,
       2.02199228e-02, 7.92317940e-04, 4.56055299e-03, 5.57995821e-04,
       4.78700630e-04, 3.42492400e-04, 9.83732939e-04, 4.60169183e-04,
       6.02096419e-04, 1.69953029e-03, 4.71636494e-04, 4.61407254e-04,
       2.67645972e-03, 5.45862347e-03, 7.71917457e-04, 1.34210533e-03,
       1.03829418e-02, 6.80623431e-04, 9.44982666e-03, 4.95269719e-03,
       2.20453066e-01, 9.10046114e-04, 1.58227505e-05, 9.75562496e-03,
       4.32497780e-02, 9.80924519e-05, 1.05137128e-04, 1.38389582e-04,
      

In [11]:
top_n = 31
selected_indices = np.argsort(feature_importances)[-top_n:]

# Sort the selected indices so that they are in the original order of features
selected_indices = np.sort(selected_indices)

print(f"Number of features selected: {len(selected_indices)}")

Number of features selected: 31


In [12]:
X_train_selected = X_train[:, selected_indices]
X_test_selected = X_test[:, selected_indices]
X_val_selected = X_val[:, selected_indices]

In [13]:
y_train = np.where(y_train == 4, 0, 1)
y_test = np.where(y_test == 4, 0, 1)
y_val = np.where(y_val == 4, 0, 1)

# PyDeepInsight

In [14]:
from pyDeepInsight import ImageTransformer

In [15]:
it = ImageTransformer(
    pixels=8,
    feature_extractor='tsne',
    discretization='lsa'
)

In [16]:
it.fit(X_train_selected)
X_train_images = it.transform(X_train_selected, 'pytorch')

X_test_images = it.transform(X_test_selected, 'pytorch')

X_val_images = it.transform(X_val_selected, 'pytorch')

# MAE

In [17]:
num_samples, channels, img_height, img_width = X_train_images.shape
latent_dim = 16

In [18]:
class Encoder(nn.Module):
    def __init__(self, img_channels=1, feature_dim=32, latent_dim=2):
        super(Encoder, self).__init__()
        self.conv1 = nn.Conv2d(img_channels, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(32 * 2 * 2, feature_dim)

    def forward(self, x):
        x = self.relu(self.conv1(x))  # Output: (batch_size, 16, 8, 8)
        x = self.pool(x)              # Output: (batch_size, 16, 4, 4)
        x = self.relu(self.conv2(x))  # Output: (batch_size, 32, 4, 4)
        x = self.pool(x)              # Output: (batch_size, 32, 2, 2)
        x = x.view(x.size(0), -1)     # Flatten to (batch_size, 128)
        x = self.fc1(x)               # Output: (batch_size, feature_dim)
        return x

In [19]:
class Decoder(nn.Module):
    def __init__(self, img_channels=1, feature_dim=32):
        super(Decoder, self).__init__()
        self.fc2 = nn.Linear(feature_dim, 32 * 2 * 2)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

        self.deconv1 = nn.ConvTranspose2d(32, 16, kernel_size=3, stride=1, padding=1)
        self.deconv2 = nn.ConvTranspose2d(16, img_channels, kernel_size=3, stride=1, padding=1)
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')

    def forward(self, z):
        x = self.relu(self.fc2(z))           # Output: (batch_size, 128)
        x = x.view(x.size(0), 32, 2, 2)      # Reshape to (batch_size, 32, 2, 2)
        x = self.upsample(x)                 # Upsample to (batch_size, 32, 4, 4)
        x = self.relu(self.deconv1(x))       # Output: (batch_size, 16, 4, 4)
        x = self.upsample(x)                 # Upsample to (batch_size, 16, 8, 8)
        x = self.sigmoid(self.deconv2(x))    # Output: (batch_size, img_channels, 8, 8)
        return x

In [20]:
class MAE(nn.Module):
    def __init__(self, img_channels=1, feature_dim=32, latent_dim=2):
        super(MAE, self).__init__()
        self.encoder = Encoder(img_channels, feature_dim, latent_dim)
        self.decoder = Decoder(img_channels, feature_dim)

    def mask_input(self, x, mask_ratio=0.25):
        # Generate a mask with 0s and 1s, keeping only (1-mask_ratio) of the original input
        mask = torch.rand(x.shape, device=x.device) > mask_ratio
        x_masked = x * mask
        return x_masked, mask

    def forward(self, x, mask_ratio=0.25):
        x_masked, mask = self.mask_input(x, mask_ratio)  # Apply masking to input
        z = self.encoder(x_masked)
        reconstructed = self.decoder(z)
        return reconstructed, mask

In [21]:
def mae_loss_function(reconstructed, original, mask):
    # Only calculate reconstruction loss on the masked parts
    masked_original = original * mask
    reconstruction_loss = F.mse_loss(reconstructed, masked_original, reduction='sum')
    return reconstruction_loss

In [22]:
model = MAE(img_channels=3, feature_dim=32, latent_dim=16).to(device)
model.load_state_dict(torch.load("deepinsight_mae_normal.pth"))

<All keys matched successfully>

# Extract

In [23]:
normal_indices = np.where(y_train == 0)[0]
X_train_normal = X_train_images[normal_indices]
y_train_normal = y_train[normal_indices]

X_train_tensor = torch.tensor(X_train_normal, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_images, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val_images, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor)
test_dataset = TensorDataset(X_test_tensor, torch.tensor(y_test, dtype=torch.long))
val_dataset = TensorDataset(X_val_tensor, torch.tensor(y_val, dtype=torch.long))

batch_size = 32 
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

/tmp/ipykernel_165/2967385897.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_tensor = torch.tensor(X_train_normal, dtype=torch.float32)
/tmp/ipykernel_165/2967385897.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(X_test_images, dtype=torch.float32)
/tmp/ipykernel_165/2967385897.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val_tensor = torch.tensor(X_val_images, dtype=torch.float32)


In [24]:
def extract_latent_features(model, data_loader, device='cuda'):
    model.eval() 
    latent_features = []  

    with torch.no_grad():
        for batch in tqdm(data_loader, total=len(data_loader), desc="Extracting features"):
            if len(batch) == 2:
                data, _ = batch  
            else:
                (data,) = batch  
            
            data = data.to(device)

            latent_feature = model.encoder(data)
            latent_features.append(latent_feature.cpu().numpy())

    latent_features = np.concatenate(latent_features, axis=0)
    
    return latent_features


In [25]:
train_latent_features = extract_latent_features(model, train_loader, device)
val_latent_features = extract_latent_features(model, val_loader, device)
test_latent_features = extract_latent_features(model, test_loader, device)

Extracting features: 100%|██████████| 3682/3682 [00:03<00:00, 1194.01it/s]


# SGDOCSVM

In [29]:
from sklearn.linear_model import SGDOneClassSVM

In [34]:
def objective(trial):

    nu = trial.suggest_float('nu', 0.01, 0.5)  
    learning_rate = trial.suggest_categorical('learning_rate', ['constant', 'optimal', 'invscaling', 'adaptive'])
    eta0 = trial.suggest_float('eta0', 1e-6, 0.5)
    power_t = trial.suggest_float('power_t', -3, 3)

    sgdocsvm = SGDOneClassSVM(nu=nu, learning_rate=learning_rate, eta0=eta0, power_t=power_t)
    sgdocsvm.fit(train_latent_features)

    val_predictions = sgdocsvm.predict(val_latent_features)
    val_true_labels = np.where(y_val == 0, 1, -1)

    f1 = f1_score(val_true_labels, val_predictions, pos_label=1, average='binary')

    return f1 

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

best_params = study.best_params
print(f"Best Hyperparameters: {best_params}")

[I 2024-10-06 21:49:16,302] A new study created in memory with name: no-name-2f23dbef-df18-42e7-a055-e3eb77ab15e1
[I 2024-10-06 21:49:16,406] Trial 0 finished with value: 0.4018367538187611 and parameters: {'nu': 0.13533588259048898, 'learning_rate': 'constant', 'eta0': 0.2606531130399098, 'power_t': 1.8124905047071218}. Best is trial 0 with value: 0.4018367538187611.
[I 2024-10-06 21:49:16,511] Trial 1 finished with value: 0.3402415916627191 and parameters: {'nu': 0.2964838317451876, 'learning_rate': 'invscaling', 'eta0': 0.11711903193790155, 'power_t': 0.43929890351241063}. Best is trial 0 with value: 0.4018367538187611.
[I 2024-10-06 21:49:16,611] Trial 2 finished with value: 0.23798792756539236 and parameters: {'nu': 0.07893476787233185, 'learning_rate': 'optimal', 'eta0': 0.33172961198079504, 'power_t': 0.8589611793227387}. Best is trial 0 with value: 0.4018367538187611.
[I 2024-10-06 21:49:16,684] Trial 3 finished with value: 0.06120769468161712 and parameters: {'nu': 0.167836044

Best Hyperparameters: {'nu': 0.47562102961452357, 'learning_rate': 'adaptive', 'eta0': 0.23508170305633402, 'power_t': 2.7128925959926202}


In [35]:
best_sgdocsvm = SGDOneClassSVM(nu=best_params['nu'], 
                               learning_rate=best_params['learning_rate'], 
                               eta0=best_params['eta0'], 
                               random_state=42)

best_sgdocsvm.fit(train_latent_features)

test_predictions = best_sgdocsvm.predict(test_latent_features)
test_true_labels = np.where(y_test == 0, 1, -1)

test_accuracy = accuracy_score(test_true_labels, test_predictions)
test_precision = precision_score(test_true_labels, test_predictions, pos_label=1, average='binary')
test_recall = recall_score(test_true_labels, test_predictions, pos_label=1, average='binary')
test_f1 = f1_score(test_true_labels, test_predictions, pos_label=1, average='binary')

print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test F1 Score: {test_f1:.4f}")

Test Accuracy: 0.9612
Test Precision: 0.4183
Test Recall: 0.5861
Test F1 Score: 0.4882


# LOF

In [36]:
from sklearn.neighbors import LocalOutlierFactor

In [37]:
def objective(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 5, 50)  
    leaf_size = trial.suggest_int('leaf_size', 20, 50) 
    metric = trial.suggest_categorical('metric', ['euclidean', 'manhattan', 'chebyshev', 'minkowski'])  

    lof = LocalOutlierFactor(n_neighbors=n_neighbors, leaf_size=leaf_size, metric=metric, novelty=True)
    lof.fit(train_latent_features)

    val_predictions = lof.predict(val_latent_features)
    val_true_labels = np.where(y_val == 0, 1, -1)

    f1 = f1_score(val_true_labels, val_predictions, pos_label=1, average='binary')

    return f1

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

best_params = study.best_params
print(f"Best Hyperparameters: {best_params}")

[I 2024-10-06 21:50:15,977] A new study created in memory with name: no-name-5c2db3fb-0f3a-4f48-a917-426f334bc6dc
[I 2024-10-06 21:50:20,007] Trial 0 finished with value: 0.9283320031923384 and parameters: {'n_neighbors': 24, 'leaf_size': 38, 'metric': 'chebyshev'}. Best is trial 0 with value: 0.9283320031923384.
[I 2024-10-06 21:50:21,248] Trial 1 finished with value: 0.9879231161762204 and parameters: {'n_neighbors': 36, 'leaf_size': 39, 'metric': 'minkowski'}. Best is trial 1 with value: 0.9879231161762204.
[I 2024-10-06 21:50:22,437] Trial 2 finished with value: 0.9874063989108237 and parameters: {'n_neighbors': 34, 'leaf_size': 33, 'metric': 'minkowski'}. Best is trial 1 with value: 0.9879231161762204.
[I 2024-10-06 21:50:23,652] Trial 3 finished with value: 0.9767441860465116 and parameters: {'n_neighbors': 30, 'leaf_size': 38, 'metric': 'euclidean'}. Best is trial 1 with value: 0.9879231161762204.
[I 2024-10-06 21:50:27,632] Trial 4 finished with value: 0.9879272232613501 and pa

Best Hyperparameters: {'n_neighbors': 16, 'leaf_size': 43, 'metric': 'chebyshev'}


In [38]:
best_lof = LocalOutlierFactor(n_neighbors=best_params['n_neighbors'], 
                              leaf_size=best_params['leaf_size'], 
                              metric=best_params['metric'], 
                              novelty=True)

best_lof.fit(train_latent_features)

test_predictions = best_lof.predict(test_latent_features)
test_true_labels = np.where(y_test == 0, 1, -1)

test_accuracy = accuracy_score(test_true_labels, test_predictions)
test_precision = precision_score(test_true_labels, test_predictions, pos_label=1, average='binary')
test_recall = recall_score(test_true_labels, test_predictions, pos_label=1, average='binary')
test_f1 = f1_score(test_true_labels, test_predictions, pos_label=1, average='binary')

print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test F1 Score: {test_f1:.4f}")

Test Accuracy: 0.9993
Test Precision: 1.0000
Test Recall: 0.9766
Test F1 Score: 0.9882


# IF

In [39]:
from sklearn.ensemble import IsolationForest

In [40]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)  
    max_samples = trial.suggest_float('max_samples', 0.5, 1.0)  
    contamination = trial.suggest_float('contamination', 0.01, 0.5) 
    max_features = trial.suggest_float('max_features', 0.5, 1.0)

    iso_forest = IsolationForest(n_estimators=n_estimators, 
                                 max_samples=max_samples, 
                                 contamination=contamination, 
                                 max_features=max_features, 
                                 random_state=42)

    iso_forest.fit(train_latent_features)
    
    val_predictions = iso_forest.predict(val_latent_features)
    val_true_labels = np.where(y_val == 0, 1, -1)

    f1 = f1_score(val_true_labels, val_predictions, pos_label=1, average='binary')
    return f1

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Get the best hyperparameters
best_params = study.best_params
print(f"Best Hyperparameters: {best_params}")

[I 2024-10-06 21:54:30,124] A new study created in memory with name: no-name-84192326-0c20-4bf9-958a-27938b42298d
[I 2024-10-06 21:54:32,069] Trial 0 finished with value: 0.758347245409015 and parameters: {'n_estimators': 90, 'max_samples': 0.6522849072667373, 'contamination': 0.36391882009091236, 'max_features': 0.9862815306458456}. Best is trial 0 with value: 0.758347245409015.
[I 2024-10-06 21:54:36,835] Trial 1 finished with value: 0.7984652665589661 and parameters: {'n_estimators': 294, 'max_samples': 0.6358450534687283, 'contamination': 0.3178252827344318, 'max_features': 0.5760135291257376}. Best is trial 1 with value: 0.7984652665589661.
[I 2024-10-06 21:54:42,756] Trial 2 finished with value: 0.9739610277634075 and parameters: {'n_estimators': 279, 'max_samples': 0.65011658278168, 'contamination': 0.041538543008933435, 'max_features': 0.9843608667509698}. Best is trial 2 with value: 0.9739610277634075.
[I 2024-10-06 21:54:45,659] Trial 3 finished with value: 0.9671931956257594

Best Hyperparameters: {'n_estimators': 250, 'max_samples': 0.8517257351186874, 'contamination': 0.016324198821552595, 'max_features': 0.6936589802865923}


In [41]:
best_iso_forest = IsolationForest(n_estimators=best_params['n_estimators'], 
                                  max_samples=best_params['max_samples'], 
                                  contamination=best_params['contamination'], 
                                  max_features=best_params['max_features'], 
                                  random_state=42)

best_iso_forest.fit(train_latent_features)
test_predictions = best_iso_forest.predict(test_latent_features)
test_true_labels = np.where(y_test == 0, 1, -1)

test_accuracy = accuracy_score(test_true_labels, test_predictions)
test_precision = precision_score(test_true_labels, test_predictions, pos_label=1, average='binary')
test_recall = recall_score(test_true_labels, test_predictions, pos_label=1, average='binary')
test_f1 = f1_score(test_true_labels, test_predictions, pos_label=1, average='binary')

print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test F1 Score: {test_f1:.4f}")

Test Accuracy: 0.9994
Test Precision: 1.0000
Test Recall: 0.9809
Test F1 Score: 0.9904


# PCA Reconstruction

In [26]:
from sklearn.decomposition import PCA

In [27]:
def reconstruction_error(X, pca):
    X_pca = pca.transform(X)
    X_reconstructed = pca.inverse_transform(X_pca)
    return np.mean((X - X_reconstructed) ** 2, axis=1)
    
def objective(trial):
    n_components = trial.suggest_int('n_components', 2, min(train_latent_features.shape[1], 50))
    percentile = trial.suggest_float('percentile', 90.0, 99.9)  

    pca = PCA(n_components=n_components, whiten=True, svd_solver='auto')

    normal_data_pca = pca.fit_transform(train_latent_features)
    normal_data_reconstructed = pca.inverse_transform(normal_data_pca)

    reconstruction_errors = np.mean((train_latent_features - normal_data_reconstructed) ** 2, axis=1)
    threshold = np.percentile(reconstruction_errors, percentile)
    val_reconstruction_errors = reconstruction_error(val_latent_features, pca)
    val_predictions = np.where(val_reconstruction_errors > threshold, -1, 1)

    val_true_labels = np.where(y_val == 0, 1, -1)

    f1 = f1_score(val_true_labels, val_predictions, pos_label=1, average='binary')

    return f1

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

best_params = study.best_params
print(f"Best Hyperparameters: {best_params}")

[I 2024-10-06 21:45:07,540] A new study created in memory with name: no-name-4b18c0c0-f91b-4f6a-bc0e-8ce6082fa2cd
[I 2024-10-06 21:45:08,024] Trial 0 finished with value: 0.9951021786860328 and parameters: {'n_components': 12, 'percentile': 99.0548960557335}. Best is trial 0 with value: 0.9951021786860328.
[I 2024-10-06 21:45:08,170] Trial 1 finished with value: 0.9838414126270199 and parameters: {'n_components': 30, 'percentile': 99.31137426890074}. Best is trial 0 with value: 0.9951021786860328.
[I 2024-10-06 21:45:08,447] Trial 2 finished with value: 0.9581509367886535 and parameters: {'n_components': 10, 'percentile': 92.45417905922538}. Best is trial 0 with value: 0.9951021786860328.
[I 2024-10-06 21:45:08,630] Trial 3 finished with value: 0.9427860696517413 and parameters: {'n_components': 11, 'percentile': 90.19347730839459}. Best is trial 0 with value: 0.9951021786860328.
[I 2024-10-06 21:45:08,757] Trial 4 finished with value: 0.9677307425399029 and parameters: {'n_components'

Best Hyperparameters: {'n_components': 23, 'percentile': 99.88986337834315}


In [28]:
best_pca = PCA(n_components=best_params['n_components'], whiten=True, svd_solver='auto')

normal_data_pca = best_pca.fit_transform(train_latent_features)
normal_data_reconstructed = best_pca.inverse_transform(normal_data_pca)

reconstruction_errors = np.mean((train_latent_features - normal_data_reconstructed) ** 2, axis=1)

threshold = np.percentile(reconstruction_errors, best_params['percentile'])

test_pca = best_pca.transform(test_latent_features)
test_reconstructed = best_pca.inverse_transform(test_pca)
test_reconstruction_errors = np.mean((test_latent_features - test_reconstructed) ** 2, axis=1)

test_predictions = np.where(test_reconstruction_errors > threshold, -1, 1) 

test_true_labels = np.where(y_test == 0, 1, -1)

test_accuracy = accuracy_score(test_true_labels, test_predictions)
test_precision = precision_score(test_true_labels, test_predictions, pos_label=1, average='binary')
test_recall = recall_score(test_true_labels, test_predictions, pos_label=1, average='binary')
test_f1 = f1_score(test_true_labels, test_predictions, pos_label=1, average='binary')

print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test F1 Score: {test_f1:.4f}")

Test Accuracy: 1.0000
Test Precision: 1.0000
Test Recall: 0.9992
Test F1 Score: 0.9996


# SLAD

In [43]:
from deepod.models.tabular import SLAD

In [44]:
def objective(trial):
    epochs = trial.suggest_int('epochs', 1, 20)
    hidden_dims = trial.suggest_int('hidden_dims', 50, 150)
    n_slad_ensemble = trial.suggest_int('n_slad_ensemble', 10, 50)

    slad_model = SLAD(epochs=epochs,
                      hidden_dims=hidden_dims,
                      n_slad_ensemble=n_slad_ensemble,
                      random_state=42)

    slad_model.fit(train_latent_features)
    
    val_predictions = slad_model.predict(val_latent_features)
    val_true_labels = np.where(y_val == 0, 0, 1)
    
    f1 = f1_score(val_true_labels, val_predictions, pos_label=0, average='binary')
    return f1

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

best_params = study.best_params
print(f"Best Hyperparameters: {best_params}")

[I 2024-10-06 22:09:17,097] A new study created in memory with name: no-name-5638df5c-6ce7-4c30-a734-d1db508714eb


Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 32, ensemble size: 18
len pool: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32]
epoch  1, training loss: 0.546880, time: 0.8s
epoch 10, training loss: 0.525268, time: 0.5s
Start Inference on the training data...


[I 2024-10-06 22:10:16,255] Trial 0 finished with value: 0.9420341394025604 and parameters: {'epochs': 19, 'hidden_dims': 85, 'n_slad_ensemble': 18}. Best is trial 0 with value: 0.9420341394025604.


Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 32, ensemble size: 20
len pool: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32]
epoch  1, training loss: 0.574282, time: 0.5s
epoch 10, training loss: 0.555246, time: 0.5s
Start Inference on the training data...


[I 2024-10-06 22:11:27,673] Trial 1 finished with value: 0.9416577730345073 and parameters: {'epochs': 19, 'hidden_dims': 121, 'n_slad_ensemble': 20}. Best is trial 0 with value: 0.9420341394025604.


...

Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 32, ensemble size: 35
len pool: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32]
epoch  1, training loss: 0.612610, time: 0.6s
epoch 10, training loss: 0.586125, time: 0.6s
Start Inference on the training data...


[I 2024-10-06 22:49:54,456] Trial 24 finished with value: 0.9414694894146949 and parameters: {'epochs': 17, 'hidden_dims': 100, 'n_slad_ensemble': 35}. Best is trial 2 with value: 0.9450354609929078.


Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 32, ensemble size: 28
len pool: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32]
epoch  1, training loss: 0.604424, time: 0.6s
epoch 10, training loss: 0.582741, time: 0.5s
Start Inference on the training data...


[W 2024-10-06 22:50:39,122] Trial 25 failed with parameters: {'epochs': 11, 'hidden_dims': 136, 'n_slad_ensemble': 28} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_165/313744358.py", line 13, in objective
    val_predictions = slad_model.predict(val_latent_features)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/deepod/core/base_model.py", line 264, in predict
    pred_score = self.decision_function(X)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/deepod/models/tabular/slad.py", line 118, in decision_function
    for batch_x in test_loader:
  File "/opt/conda/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 631, in __next__
    self

KeyboardInterrupt: 

In [46]:
best_slad_model = SLAD(epochs=best_params['epochs'],
                      hidden_dims=best_params['hidden_dims'],
                      n_slad_ensemble=best_params['n_slad_ensemble'],
                      random_state=42)

best_slad_model.fit(train_latent_features)
test_predictions = best_slad_model.predict(test_latent_features)
test_true_labels = np.where(y_test == 0, 0, 1)

test_accuracy = accuracy_score(test_true_labels, test_predictions)
test_precision = precision_score(test_true_labels, test_predictions, pos_label=1, average='binary')
test_recall = recall_score(test_true_labels, test_predictions, pos_label=1, average='binary')
test_f1 = f1_score(test_true_labels, test_predictions, pos_label=1, average='binary')

print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test F1 Score: {test_f1:.4f}")

Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 32, ensemble size: 46
len pool: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32]
epoch  1, training loss: 0.604720, time: 0.5s
epoch 10, training loss: 0.575029, time: 0.6s
Start Inference on the training data...
Test Accuracy: 0.9967
Test Precision: 0.9966
Test Recall: 1.0000
Test F1 Score: 0.9983


# ICL

In [57]:
from deepod.models.tabular import ICL

In [58]:
def objective(trial):
    epochs = trial.suggest_int('epochs', 1, 20)
    rep_dim = trial.suggest_int('rep_dim', 50, 150)
    temperature = trial.suggest_float('temperature', 0.001, 0.1)

    icl_model = ICL(epochs=epochs,
                      rep_dim=rep_dim,
                      temperature=temperature,
                      random_state=42)

    icl_model.fit(train_latent_features)
    
    val_predictions = icl_model.predict(val_latent_features)
    val_true_labels = np.where(y_val == 0, 0, 1)
    
    f1 = f1_score(val_true_labels, val_predictions, pos_label=0, average='binary')
    return f1

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

best_params = study.best_params
print(f"Best Hyperparameters: {best_params}")

[I 2024-10-06 23:23:19,739] A new study created in memory with name: no-name-e8bc4f88-11e5-4639-9142-9081f1e932a0


Start Training...
ensemble size: 3
kernel size: 2
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=30, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(31, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(31, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=81, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(31, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=2, out_features=50, bias=Fal

testing: 100%|██████████| 1473/1473 [00:03<00:00, 426.26it/s]
[I 2024-10-06 23:24:22,499] Trial 0 finished with value: 0.9472753007784855 and parameters: {'epochs': 17, 'rep_dim': 81, 'temperature': 0.0760299845415369}. Best is trial 0 with value: 0.9472753007784855.


Start Training...
ensemble size: 3
kernel size: 2
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=30, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(31, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(31, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=107, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(31, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=2, out_features=50, bias=Fa

testing: 100%|██████████| 1473/1473 [00:03<00:00, 430.39it/s]
[I 2024-10-06 23:25:16,293] Trial 1 finished with value: 0.9416577730345073 and parameters: {'epochs': 14, 'rep_dim': 107, 'temperature': 0.07259122025301018}. Best is trial 0 with value: 0.9472753007784855.


...

Start Training...
ensemble size: 3
kernel size: 2
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=30, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(31, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(31, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=72, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(31, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=2, out_features=50, bias=Fal

testing: 100%|██████████| 1473/1473 [00:03<00:00, 429.40it/s]
[I 2024-10-06 23:38:03,378] Trial 18 finished with value: 0.9442867281760113 and parameters: {'epochs': 1, 'rep_dim': 72, 'temperature': 0.06409707765220436}. Best is trial 0 with value: 0.9472753007784855.


Start Training...
ensemble size: 3
kernel size: 2
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=30, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(31, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(31, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=51, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(31, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=2, out_features=50, bias=Fal

testing: 100%|██████████| 1473/1473 [00:03<00:00, 430.88it/s]
[I 2024-10-06 23:38:18,528] Trial 19 finished with value: 0.9457831325301205 and parameters: {'epochs': 1, 'rep_dim': 51, 'temperature': 0.08336224816985124}. Best is trial 0 with value: 0.9472753007784855.


Best Hyperparameters: {'epochs': 17, 'rep_dim': 81, 'temperature': 0.0760299845415369}


In [59]:
best_icl_model = ICL(epochs=best_params['epochs'],
                      rep_dim=best_params['rep_dim'],
                      temperature=best_params['temperature'],
                      random_state=42)

best_icl_model.fit(train_latent_features)
test_predictions = best_icl_model.predict(test_latent_features)
test_true_labels = np.where(y_test == 0, 0, 1)

test_accuracy = accuracy_score(test_true_labels, test_predictions)
test_precision = precision_score(test_true_labels, test_predictions, pos_label=1, average='binary')
test_recall = recall_score(test_true_labels, test_predictions, pos_label=1, average='binary')
test_f1 = f1_score(test_true_labels, test_predictions, pos_label=1, average='binary')

print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test F1 Score: {test_f1:.4f}")

Start Training...
ensemble size: 3
kernel size: 2
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=30, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(31, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(31, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=81, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(31, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=2, out_features=50, bias=Fal

testing: 100%|██████████| 1841/1841 [00:04<00:00, 431.46it/s]


Test Accuracy: 0.9966
Test Precision: 0.9965
Test Recall: 1.0000
Test F1 Score: 0.9983


# RCA

In [47]:
from deepod.models.tabular import RCA

In [48]:
def objective(trial):
    epochs = trial.suggest_int('epochs', 1, 20)
    rep_dim = trial.suggest_int('rep_dim', 50, 150)
    anom_ratio = trial.suggest_float('anom_ratio', 0.01, 0.1)

    rca_model = RCA(epochs=epochs,
                      rep_dim=rep_dim,
                      anom_ratio=anom_ratio,
                      random_state=42)

    rca_model.fit(train_latent_features)
    
    val_predictions = rca_model.predict(val_latent_features)
    val_true_labels = np.where(y_val == 0, 0, 1)
    
    f1 = f1_score(val_true_labels, val_predictions, pos_label=0, average='binary')
    return f1

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

best_params = study.best_params
print(f"Best Hyperparameters: {best_params}")

[I 2024-10-06 22:55:43,908] A new study created in memory with name: no-name-86e67b36-3a9b-4ed0-9276-c1c74fa4b4ca


Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=32, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=67, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=32, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False

100%|██████████| 10/10 [00:25<00:00,  2.53s/it]
[I 2024-10-06 22:56:27,602] Trial 0 finished with value: 0.7559413690052654 and parameters: {'epochs': 14, 'rep_dim': 67, 'anom_ratio': 0.01888044322352994}. Best is trial 0 with value: 0.7559413690052654.


Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=32, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=104, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=32, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=Fals

100%|██████████| 10/10 [00:22<00:00,  2.22s/it]
[I 2024-10-06 22:57:12,413] Trial 1 finished with value: 0.6529569238257483 and parameters: {'epochs': 19, 'rep_dim': 104, 'anom_ratio': 0.07433194622437658}. Best is trial 0 with value: 0.7559413690052654.


...

Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=32, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=118, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=32, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=Fals

100%|██████████| 10/10 [00:22<00:00,  2.23s/it]
[I 2024-10-06 23:08:02,432] Trial 18 finished with value: 0.7688740762208376 and parameters: {'epochs': 15, 'rep_dim': 118, 'anom_ratio': 0.021220360080557436}. Best is trial 12 with value: 0.7929502369668247.


Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=32, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=93, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=32, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False

100%|██████████| 10/10 [00:22<00:00,  2.23s/it]
[I 2024-10-06 23:08:28,634] Trial 19 finished with value: 0.22894242268921922 and parameters: {'epochs': 1, 'rep_dim': 93, 'anom_ratio': 0.039056344693944894}. Best is trial 12 with value: 0.7929502369668247.


Best Hyperparameters: {'epochs': 20, 'rep_dim': 89, 'anom_ratio': 0.01598779755295056}


In [49]:
best_rca_model = RCA(epochs=best_params['epochs'],
                      rep_dim=best_params['rep_dim'],
                      anom_ratio=best_params['anom_ratio'],
                      random_state=42)

best_rca_model.fit(train_latent_features)
test_predictions = best_rca_model.predict(test_latent_features)
test_true_labels = np.where(y_test == 0, 0, 1)

test_accuracy = accuracy_score(test_true_labels, test_predictions)
test_precision = precision_score(test_true_labels, test_predictions, pos_label=1, average='binary')
test_recall = recall_score(test_true_labels, test_predictions, pos_label=1, average='binary')
test_f1 = f1_score(test_true_labels, test_predictions, pos_label=1, average='binary')

print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test F1 Score: {test_f1:.4f}")

Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=32, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=89, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=32, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False

100%|██████████| 10/10 [00:27<00:00,  2.76s/it]

Test Accuracy: 0.9849
Test Precision: 0.9965
Test Recall: 0.9879
Test F1 Score: 0.9922


# RDP

In [54]:
from deepod.models.tabular import RDP

In [55]:
def objective(trial):
    epochs = trial.suggest_int('epochs', 1, 20)
    rep_dim = trial.suggest_int('rep_dim', 50, 150)

    rdp_model = RDP(epochs=epochs,
                      rep_dim=rep_dim,
                      random_state=42)

    rdp_model.fit(train_latent_features)
    
    val_predictions = rdp_model.predict(val_latent_features)
    val_true_labels = np.where(y_val == 0, 0, 1)
    
    f1 = f1_score(val_true_labels, val_predictions, pos_label=0, average='binary')
    return f1

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

best_params = study.best_params
print(f"Best Hyperparameters: {best_params}")

[I 2024-10-06 23:17:49,587] A new study created in memory with name: no-name-0f409735-4133-4385-ac72-055a082a7845


Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=32, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=63, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.000019, time: 0.8s
epoch 10, training loss: 0.000012, time: 0.8s
Start Inference on the training data...


testing: 100%|██████████| 1473/1473 [00:02<00:00, 589.77it/s]
[I 2024-10-06 23:18:01,555] Trial 0 finished with value: 0.8130859764612008 and parameters: {'epochs': 11, 'rep_dim': 63}. Best is trial 0 with value: 0.8130859764612008.


Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=32, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=93, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.000017, time: 0.8s
Start Inference on the training data...


testing: 100%|██████████| 1473/1473 [00:02<00:00, 590.67it/s]
[I 2024-10-06 23:18:06,110] Trial 1 finished with value: 0.9363603861279943 and parameters: {'epochs': 2, 'rep_dim': 93}. Best is trial 1 with value: 0.9363603861279943.


...

Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=32, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=50, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.000021, time: 0.8s
Start Inference on the training data...


testing: 100%|██████████| 1473/1473 [00:02<00:00, 592.71it/s]
[I 2024-10-06 23:21:33,222] Trial 18 finished with value: 0.762993762993763 and parameters: {'epochs': 7, 'rep_dim': 50}. Best is trial 8 with value: 0.9439318665720369.


Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=32, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=134, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.000018, time: 0.8s
Start Inference on the training data...


testing: 100%|██████████| 1473/1473 [00:02<00:00, 588.43it/s]
[I 2024-10-06 23:21:38,653] Trial 19 finished with value: 0.940149625935162 and parameters: {'epochs': 3, 'rep_dim': 134}. Best is trial 8 with value: 0.9439318665720369.


Best Hyperparameters: {'epochs': 1, 'rep_dim': 112}


In [56]:
best_rdp_model = RDP(epochs=best_params['epochs'],
                      rep_dim=best_params['rep_dim'],
                      random_state=42)

best_rdp_model.fit(train_latent_features)
test_predictions = best_rdp_model.predict(test_latent_features)
test_true_labels = np.where(y_test == 0, 0, 1)

test_accuracy = accuracy_score(test_true_labels, test_predictions)
test_precision = precision_score(test_true_labels, test_predictions, pos_label=1, average='binary')
test_recall = recall_score(test_true_labels, test_predictions, pos_label=1, average='binary')
test_f1 = f1_score(test_true_labels, test_predictions, pos_label=1, average='binary')

print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test F1 Score: {test_f1:.4f}")

Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=32, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=112, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.000016, time: 0.8s
Start Inference on the training data...


testing: 100%|██████████| 1841/1841 [00:03<00:00, 585.37it/s]


Test Accuracy: 0.9964
Test Precision: 0.9963
Test Recall: 0.9999
Test F1 Score: 0.9981


# DeepSVDD

In [51]:
from deepod.models.tabular import DeepSVDD

In [52]:
def objective(trial):
    epochs = trial.suggest_int('epochs', 1, 20)
    rep_dim = trial.suggest_int('rep_dim', 50, 150)

    deepsvdd_model = DeepSVDD(epochs=epochs,
                      rep_dim=rep_dim,
                      random_state=42)

    deepsvdd_model.fit(train_latent_features)
    
    val_predictions = deepsvdd_model.predict(val_latent_features)
    val_true_labels = np.where(y_val == 0, 0, 1)
    
    f1 = f1_score(val_true_labels, val_predictions, pos_label=0, average='binary')
    return f1

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

best_params = study.best_params
print(f"Best Hyperparameters: {best_params}")

[I 2024-10-06 23:14:56,567] A new study created in memory with name: no-name-1662ce95-527e-42bc-9244-08bccdbf11de


Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=32, out_features=100, bias=False)
      (act_layer): ReLU()
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): ReLU()
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=130, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.016776, time: 0.3s
Start Inference on the training data...


testing: 100%|██████████| 1473/1473 [00:01<00:00, 1427.19it/s]
[I 2024-10-06 23:15:00,238] Trial 0 finished with value: 0.9454094292803971 and parameters: {'epochs': 7, 'rep_dim': 130}. Best is trial 0 with value: 0.9454094292803971.


Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=32, out_features=100, bias=False)
      (act_layer): ReLU()
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): ReLU()
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=112, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.013780, time: 0.3s
Start Inference on the training data...


testing: 100%|██████████| 1473/1473 [00:01<00:00, 1325.42it/s]
[I 2024-10-06 23:15:04,379] Trial 1 finished with value: 0.9412811387900356 and parameters: {'epochs': 8, 'rep_dim': 112}. Best is trial 0 with value: 0.9454094292803971.


...

Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=32, out_features=100, bias=False)
      (act_layer): ReLU()
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): ReLU()
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=75, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.008782, time: 0.3s
Start Inference on the training data...


testing: 100%|██████████| 1473/1473 [00:01<00:00, 1325.32it/s]
[I 2024-10-06 23:16:48,355] Trial 18 finished with value: 0.9418459896852214 and parameters: {'epochs': 1, 'rep_dim': 75}. Best is trial 9 with value: 0.9483916578296218.


Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=32, out_features=100, bias=False)
      (act_layer): ReLU()
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): ReLU()
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=150, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.018763, time: 0.3s
epoch 10, training loss: 0.000197, time: 0.3s
Start Inference on the training data...


testing: 100%|██████████| 1473/1473 [00:01<00:00, 1323.42it/s]
[I 2024-10-06 23:16:55,259] Trial 19 finished with value: 0.9457831325301205 and parameters: {'epochs': 17, 'rep_dim': 150}. Best is trial 9 with value: 0.9483916578296218.


Best Hyperparameters: {'epochs': 19, 'rep_dim': 128}


In [53]:
best_deepsvdd_model = DeepSVDD(epochs=best_params['epochs'],
                      rep_dim=best_params['rep_dim'],
                      random_state=42)

best_deepsvdd_model.fit(train_latent_features)
test_predictions = best_deepsvdd_model.predict(test_latent_features)
test_true_labels = np.where(y_test == 0, 0, 1)

test_accuracy = accuracy_score(test_true_labels, test_predictions)
test_precision = precision_score(test_true_labels, test_predictions, pos_label=1, average='binary')
test_recall = recall_score(test_true_labels, test_predictions, pos_label=1, average='binary')
test_f1 = f1_score(test_true_labels, test_predictions, pos_label=1, average='binary')

print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test F1 Score: {test_f1:.4f}")

Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=32, out_features=100, bias=False)
      (act_layer): ReLU()
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): ReLU()
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.016809, time: 0.3s
epoch 10, training loss: 0.000154, time: 0.3s
Start Inference on the training data...


testing: 100%|██████████| 1841/1841 [00:01<00:00, 1320.20it/s]


Test Accuracy: 0.9967
Test Precision: 0.9966
Test Recall: 1.0000
Test F1 Score: 0.9983


# NeuTraL

# GOAD

In [69]:
from deepod.models.tabular import GOAD

In [70]:
def objective(trial):
    epochs = trial.suggest_int('epochs', 1, 20)
    hidden_dim = trial.suggest_int('hidden_dim', 2, 16)
    trans_dim = trial.suggest_int('trans_dim', 16, 64)

    goad_model = GOAD(epochs=epochs,
                      hidden_dim=hidden_dim,
                      trans_dim=trans_dim,
                      random_state=42)

    goad_model.fit(train_latent_features)
    
    val_predictions = goad_model.predict(val_latent_features)
    val_true_labels = np.where(y_val == 0, 0, 1)
    
    f1 = f1_score(val_true_labels, val_predictions, pos_label=0, average='binary')
    return f1

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

best_params = study.best_params
print(f"Best Hyperparameters: {best_params}")

[I 2024-09-30 06:13:24,528] A new study created in memory with name: no-name-905e5c63-ea66-448a-b3f3-53f438593f73


Start Training...
ensemble size: 1


Could not load library libcudnn_cnn_train.so.8. Error: /opt/conda/lib/python3.11/site-packages/nvidia/cudnn/lib/libcudnn_cnn_train.so.8: undefined symbol: _ZTIN10cask_cudnn14BaseKernelInfoE, version libcudnn_cnn_infer.so.8
Could not load library libcudnn_cnn_train.so.8. Error: /opt/conda/lib/python3.11/site-packages/nvidia/cudnn/lib/libcudnn_cnn_train.so.8: undefined symbol: _ZTIN10cask_cudnn14BaseKernelInfoE, version libcudnn_cnn_infer.so.8
Could not load library libcudnn_cnn_train.so.8. Error: /opt/conda/lib/python3.11/site-packages/nvidia/cudnn/lib/libcudnn_cnn_train.so.8: undefined symbol: _ZTIN10cask_cudnn14BaseKernelInfoE, version libcudnn_cnn_infer.so.8
Could not load library libcudnn_cnn_train.so.8. Error: /opt/conda/lib/python3.11/site-packages/nvidia/cudnn/lib/libcudnn_cnn_train.so.8: undefined symbol: _ZTIN10cask_cudnn14BaseKernelInfoE, version libcudnn_cnn_infer.so.8
Could not load library libcudnn_cnn_train.so.8. Error: /opt/conda/lib/python3.11/site-packages/nvidia/cudnn/

256 transformation done
GoadNet(
  (enc): ConvNet(
    (net): Sequential(
      (0): Conv1d(38, 4, kernel_size=(1,), stride=(1,), bias=False)
      (1): LeakyReLU(negative_slope=0.01)
      (2): Conv1d(4, 4, kernel_size=(1,), stride=(1,), bias=False)
      (3): LeakyReLU(negative_slope=0.01)
      (4): Conv1d(4, 4, kernel_size=(1,), stride=(1,), bias=False)
      (5): LeakyReLU(negative_slope=0.01)
      (6): Conv1d(4, 4, kernel_size=(1,), stride=(1,), bias=False)
      (7): LeakyReLU(negative_slope=0.01)
      (8): Conv1d(4, 4, kernel_size=(1,), stride=(1,), bias=False)
      (9): LeakyReLU(negative_slope=0.01)
      (10): Conv1d(4, 4, kernel_size=(1,), stride=(1,), bias=False)
    )
  )
  (head): Sequential(
    (0): LeakyReLU(negative_slope=0.2, inplace=True)
    (1): Conv1d(4, 256, kernel_size=(1,), stride=(1,))
  )
)


RuntimeError: GET was unable to find an engine to execute this computation

In [ ]:
best_goad_model = GOAD(epochs=best_params['epochs'],
                        hidden_dim=best_params['hidden_dim'],
                        trans_dim=best_params['trans_dim'],
                        random_state=42)

best_goad_model.fit(train_latent_features)
test_predictions = best_goad_model.predict(test_latent_features)
test_true_labels = np.where(y_test == 0, 0, 1)

test_accuracy = accuracy_score(test_true_labels, test_predictions)
test_precision = precision_score(test_true_labels, test_predictions, pos_label=1, average='binary')
test_recall = recall_score(test_true_labels, test_predictions, pos_label=1, average='binary')
test_f1 = f1_score(test_true_labels, test_predictions, pos_label=1, average='binary')

print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test F1 Score: {test_f1:.4f}")

# REPEN

In [71]:
from deepod.models.tabular import REPEN

In [72]:
def objective(trial):
    epochs = trial.suggest_int('epochs', 1, 20)
    rep_dim = trial.suggest_int('rep_dim', 50, 150)

    repen_model = REPEN(epochs=epochs,
                      rep_dim=rep_dim,
                      random_state=42)

    repen_model.fit(train_latent_features)
    
    val_predictions = repen_model.predict(val_latent_features)
    val_true_labels = np.where(y_val == 0, 0, 1)
    
    f1 = f1_score(val_true_labels, val_predictions, pos_label=0, average='binary')
    return f1

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

best_params = study.best_params
print(f"Best Hyperparameters: {best_params}")

[I 2024-09-30 06:14:58,964] A new study created in memory with name: no-name-b34cc522-9d2f-435c-ab94-da5a42064906


Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=32, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=148, bias=False)
      (act_layer): Identity()
    )
  )
)


[W 2024-09-30 06:23:36,410] Trial 0 failed with parameters: {'epochs': 10, 'rep_dim': 148} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_833/3979886442.py", line 9, in objective
    repen_model.fit(train_latent_features)
  File "/opt/conda/lib/python3.11/site-packages/deepod/core/base_model.py", line 185, in fit
    self._training()
  File "/opt/conda/lib/python3.11/site-packages/deepod/core/base_model.py", line 336, in _training
    for batch_x in self.train_loader:
  File "/opt/conda/lib/python3.11/site-packages/deepod/models/tabular/repen.py", line 118, in __next__
    examples, positives, negatives = self.triplet_batch_generation()
                                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/deepod

KeyboardInterrupt: 

In [ ]:
best_repen_model = REPEN(epochs=best_params['epochs'],
                        rep_dim=best_params['rep_dim'],
                        random_state=42)

best_repen_model.fit(train_latent_features)
test_predictions = best_repen_model.predict(test_latent_features)
test_true_labels = np.where(y_test == 0, 0, 1)

test_accuracy = accuracy_score(test_true_labels, test_predictions)
test_precision = precision_score(test_true_labels, test_predictions, pos_label=1, average='binary')
test_recall = recall_score(test_true_labels, test_predictions, pos_label=1, average='binary')
test_f1 = f1_score(test_true_labels, test_predictions, pos_label=1, average='binary')

print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test F1 Score: {test_f1:.4f}")